In [ ]:
import pandas as pd
import numpy as np
from tqdm import trange

import sys
sys.path.append('../')

# Load synthetic data

Load synthetic data and process it for the knapsack algorithm. In this case the number of focal students `nFocalStudents` are the values of the problem and the number of non-focal students `nOtherStudents` are the weights of the problem.

In [ ]:
from src.d01_data.student_data_api import StudentDataApi, _block_features, _census_block_column, \
_diversity_index_features, _studentno, _diversity_index_col, _period_column, _prob_col

periods_list = ["1920"]
student_data_api = StudentDataApi()

df_students = student_data_api.get_data(periods_list)
mask = df_students[_census_block_column] == 'NaN'
df_students.drop(df_students.index[mask], inplace=True)

student_data_api.get_diversity_index(df_students)
student_data_api.get_focal_probability(df_students)

In [ ]:
np.random.seed(20210704)

df_students.loc[df_students.index, 'focal'] = np.random.binomial(1, p=df_students[_prob_col])

df_students = df_students.groupby([_period_column, _census_block_column])['focal'].agg(['sum', 'count', 'mean'])
df_students.columns = ['nFocalStudents', 'nTotalStudents', 'focalRate']
df_students['nOtherStudents'] = df_students['nTotalStudents'] - df_students['nFocalStudents']

In [ ]:
data = df_students.loc[2020, ['nFocalStudents', 'nOtherStudents', 'focalRate']]
data.describe()

# Test Knapsack Algorithm

Now we can test that our implementation of the Approximate Knapsack algorithm.

In [ ]:
from src.d04_modeling.knapsack_approx import KnapsackApprox

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [ ]:
mask = data['focalRate'] > 0.0

solver = KnapsackApprox(eps=.5, data=data.loc[mask].copy(),
                        value_col='nFocalStudents',
                        weight_col='nOtherStudents',
                        scale=False)

solver.solve()

plt.imshow(pd.DataFrame(solver.value_function).fillna(-10))
plt.show()

In the previouse cell we plot the value function of the algorithm with nan filled as 0. We know recover the solution and verify that it is feasible and optimal.

In [ ]:
total_students = (data['nFocalStudents'].values + data['nOtherStudents'].values).sum()
fp_rate = 0.1
w_max = fp_rate * total_students
v_opt, solution_set = solver.get_solution(w_max=w_max)
solution_set = pd.Index(solution_set, name=data.index.name)
results = data.loc[solution_set].sum()
assert(results['nFocalStudents'] == v_opt)
assert(results['nOtherStudents'] <= w_max)